In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
model = tf.keras.models.load_model('../input/dlcv-projekt/model-best.h5')

In [ ]:
!mkdir -p /kaggle/tmp/test_dataset/test
!cp -r ../input/plant-pathology-2021-fgvc8/test_images /kaggle/tmp/test_dataset/test

In [ ]:
from PIL import Image
import numpy as np
maxsize = (224, 224)
image = Image.open('../input/plant-pathology-2021-fgvc8/test_images/85f8cb619c66b863.jpg')
image.thumbnail(maxsize, Image.ANTIALIAS)
x = np.asarray(image)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
test_datagen = ImageDataGenerator()  # rescale=1. / 255)
test_generator = test_datagen.flow_from_directory('/kaggle/tmp/test_dataset', class_mode=None, target_size=(380, 380), shuffle=False, batch_size=128)

In [ ]:
x = model.predict(test_generator)
x

In [ ]:
#labels = ['complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab']
labels = ['complex', 'frog_eye_leaf_spot', 'powdery_mildew', 'rust', 'scab']
threshold = 0.7
f = lambda x: 1 if x > threshold else 0
g = lambda x: [labels[i] for i in x if i is not 0]

In [ ]:
import numpy as np
z = np.vectorize(f)(x)
#z = z[:,:-1] # TODO: remove this once new model is trained
z

In [ ]:
from pathlib import Path
import pandas as pd
predictions = [[labels[i] for i, j in enumerate(y) if j != 0] for y in z]
predictions_str = [' '.join(i) if i != [] else 'healthy' for i in predictions]
filenames = [Path(i).name for i in test_generator.filenames]
df = pd.DataFrame({
    'image': filenames,
    'labels': predictions_str
})
df

In [ ]:
df.to_csv('submission.csv',index=False)